In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/ML/HAI817_Projet_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ML/HAI817_Projet_test.csv')

In [ ]:
# Importation des différentes librairies, classes et fonctions utilespour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns

# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# librairies NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize 

 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
my_local_drive='/content/drive/MyDrive/ML/Prof/ML_FDS'

# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
#%cd $my_local_drive

#%pwd

# fonctions utilities (affichage, confusion, etc.)
from MyNLPUtilities import *

In [ ]:
df_all = pd.concat([df_train,df_test])


In [ ]:
# compter les valeurs manquantes dans chaque colonne
num_missing_values = df_all.isna().sum()
print(num_missing_values)

public_id      612
text             0
title           23
our rating       0
ID            1264
dtype: int64


In [ ]:
df_all = df_all[~df_all['our rating'].isin(['mixture', 'other'])]

df_all['classe'] = df_all['our rating'].map({'true': 1, 'false': 2})

<ipython-input-9-c7b19893d15a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['classe'] = df_all['our rating'].map({'true': 1, 'false': 2})


In [ ]:
print(df_all["classe"].value_counts())

2    893
1    421
Name: classe, dtype: int64


Equilibrage des classes

In [ ]:
from sklearn.utils import resample
import pandas as pd



# Séparer les classes majoritaires et minoritaires
df_majority = df_all[df_all['classe'] == 2]
df_minority = df_all[df_all['classe'] == 1]

# Sous-échantillonner la classe majoritaire
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # Échantillonnage sans remplacement
                                   n_samples=len(df_minority), # Nombre d'échantillons égal à la classe minoritaire
                                   random_state=42)  # Pour la reproductibilité

# Combiner les classes majoritaire et minoritaire
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Afficher la taille du jeu de données équilibré
print("Taille du jeu de données équilibré : ", df_balanced.shape)

df_all = df_balanced
print(df_all['classe'].value_counts())

Taille du jeu de données équilibré :  (842, 6)
2    421
1    421
Name: classe, dtype: int64


Définition de la fonction  MyCleanText

In [ ]:
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def MyCleanText(X,
 lowercase=False, # mettre en minuscule
 removestopwords=False, # supprimer les stopwords
 removedigit=False, # supprimer les nombres
 getstemmer=False, # conserver la racine des termes
 getlemmatisation=False # lematisation des termes
 ):
    
    sentence = str(X)
    
    # suppression des caractères spéciaux
    sentence = re.sub(r'[^\w\s]',' ', sentence)
    
    # suppression de tous les caractères uniques
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    
    # substitution des espaces multiples par un seul espace
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)
    
    # decoupage en mots
    tokens = word_tokenize(sentence)
    
    if lowercase:
        tokens = [token.lower() for token in tokens]

    # suppression ponctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]
    
    # suppression des tokens non alphabetique ou numerique
    words = [word for word in words if word.isalnum()]

    # suppression des tokens numerique
    if removedigit:
        words = [word for word in words if not word.isdigit()]
    
    # suppression des stopwords
    if removestopwords:
        words = [word for word in words if not word in stop_words]
    
    # lemmatisation
    if getlemmatisation:
        lemmatizer=WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word)for word in words]
    
    # racinisation
    if getstemmer:
        ps = PorterStemmer()
        words=[ps.stem(word) for word in words]

    sentence= ' '.join(words)

    return sentence


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Définition de la fonction TextNormalizer

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self,
                 removestopwords=False, # suppression des stopwords
                 lowercase=False,# passage en minuscule
                 removedigit=False, # supprimer les nombres
                 getstemmer=False,# racinisation des termes
                 getlemmatisation=False # lemmatisation des termes
                ):
        self.lowercase=lowercase
        self.getstemmer=getstemmer
        self.removestopwords=removestopwords
        self.getlemmatisation=getlemmatisation
        self.removedigit=removedigit
    
    def transform(self, X, **transform_params):
        # Nettoyage du texte
        X=X.copy() # pour conserver le fichier d'origine
        return [MyCleanText(text, lowercase=self.lowercase,
                            getstemmer=self.getstemmer,
                            removestopwords=self.removestopwords,
                            getlemmatisation=self.getlemmatisation,
                            removedigit=self.removedigit) for text in X]
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)
    
    def get_params(self, deep=True):
        return {
            'lowercase':self.lowercase,
            'getstemmer':self.getstemmer,
            'removestopwords':self.removestopwords,
            'getlemmatisation':self.getlemmatisation,
            'removedigit':self.removedigit
        }
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self


In [ ]:
!pip install -U spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch==1.10.1+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.1+cu111 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.10.1+cu111


In [ ]:
!python -m spacy download fr_core_news_sm


2023-04-29 16:18:55.121013: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 16:18:57.008651: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 23.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


Extraction des entités nommées à partir d'un corpus avec NLTK et ajout de features dans Pandas

In [ ]:
import nltk
import pandas as pd

# Télécharger les corpus et modèles nécessaires pour la reconnaissance d'entités nommées
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')



# Appliquer la reconnaissance d'entités nommées
entities = []
for text in df_all.text:
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    entities_list = []
    for chunk in nltk.ne_chunk(tagged):
        if hasattr(chunk, 'label'):
            entities_list.append(' '.join(c[0] for c in chunk))
    entities.append(entities_list)

# Ajouter les entités extraites comme features
df_all['entities'] = entities


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
df_all.head(10)

,public_id,text,title,our rating,ID,classe,entities
186,NaN,"During the predawn hours on New Year’s Day, De...",Delta Force Raids Biden Compound in Ukraine,false,105625103119574386858328487241855757744,2,"[New, Delta Force, Ukraine, Mariupol, White Ho..."
833,f2fcb445,LONG-TERM MASK USE MAY CONTRIBUTE TO ADVANCED ...,FIRST CLASS Schools reopen with kids in masks ...,false,NaN,2,"[CONTRIBUTE, LUNG, STUDY]"
1010,8469a163,The Biden administration will soon deliver a s...,U.S. Senator Jeff Merkley of Oregon,false,NaN,2,"[Biden, SolarWinds, Russia, Washington, US Nat..."
200,NaN,"As many as 45,000 people may have died from th...",Top Cardiologist: Game-Changing Study Shows CO...,false,200049446239665258541118918005789068501,2,"[mRNA, COVID, Peter McCullough, COVID, McCullo..."
58,ae596f59,A Proof has recently appeared that cash attach...,The New York Times Affirms: ‘Soros & Clinton P...,false,NaN,2,"[Hillary Clinton, George Soros, Trump, New Yor..."
397,88477948,"Now that it’s 2018, an election year, I would ...",HEY LOOK! Senator Tammy Baldwin Is Back In Wis...,false,NaN,2,"[Tammy Baldwin, Wisconsin, Wisconsin, Almost, ..."
411,487c871b,Nation President Boards E-4b Command Post; Fl...,President Boards E-4b Command Post; Flying to ...,false,NaN,2,"[Nation, Boards, Command Post, NMCC Abilene, T..."
494,4905a5f5,The CO2 error is the root of the biggest scam ...,Tim Ball: The Evidence Proves That CO2 is Not ...,false,NaN,2,"[CO2, Sustainable, Intergovernmental Panel, Cl..."
285,dca71385,Sharon Stone smoking in Basic Instinct Every ...,SHOCKING REPORT: Smoking May Protect Lungs Fro...,false,NaN,2,"[Sharon, Basic, Orwellian, American, American,..."
180,0a7d532e,The Centers for Disease Control and Prevention...,CDC Announces That Students May Be Kept From P...,false,NaN,2,"[Centers, Disease Control, Tell, CDC, Sham, Wa..."


Corrélation entre la présence d'entités nommées dans les articles et la véracité des informations qu'ils contiennent

In [ ]:
# Sélectionner uniquement les lignes avec des fake news
df_fake = df_all[df_all['classe'] == 2]

# Transformer la colonne entities en une colonne avec une entité par ligne
df_fake_exploded = df_fake.explode('entities')

# Compter le nombre de fake news pour chaque entité
fake_news_count = df_fake_exploded.groupby('entities')['classe'].count().sort_values(ascending=False)

# Afficher les résultats
print(fake_news_count)


entities
Trump              397
China              238
U.S.               184
American           163
US                 144
                  ... 
Huawei Cloud         1
Hubei                1
Hubei Institute      1
Hull                 1
Koran                1
Name: classe, Length: 5973, dtype: int64


Classification de la véracité des informations à partir d'un corpus contenant des entités nommées en utilisant une SVM et la vectorisation TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import pandas as pd

# Define the pipeline
pipe = Pipeline([
    ("cleaner", TextNormalizer(removestopwords=True, lowercase=True, removedigit=True, getstemmer=False, getlemmatisation=False)),
    ("TfidfVectorizer", TfidfVectorizer()), 
    ("svm", SVC(C=1, gamma=0.001, kernel='linear'))
])



X = []
for entities in df_all.entities:
    X.append(' '.join(entities))
y = df_all['classe']


# diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ajuster le pipeline sur les données d'entraînement
pipe.fit(X_train, y_train)


from sklearn.metrics import accuracy_score

# prédire les étiquettes de classe sur l'ensemble de données de test

y_true, y_pred = y_test, pipe.predict(X_test)

print(classification_report(y_true, y_pred))

# calculer la précision en comparant les prédictions avec les étiquettes de classe réelles
accuracy = accuracy_score(y_test, y_pred)

# afficher la précision
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           1       0.73      0.78      0.75        85
           2       0.76      0.70      0.73        84

    accuracy                           0.74       169
   macro avg       0.74      0.74      0.74       169
weighted avg       0.74      0.74      0.74       169

Accuracy: 0.7396449704142012
